# Instructions

Run each cell one at a time by pressing the 'run' button above, or shift+enter when the cursor is in the cell.

First we install the PyChess library - 
"python-chess is a pure Python chess library with move generation, move validation and support for common formats"

As well as some other needed functions.

In [24]:
#PyChess library installation
import sys
!{sys.executable} -m pip install python-chess

#Import needed functions/libraries
import chess
import random
import chess.svg as render
import time

We can now use the data structures from this library to create the 'search' and 'evaluation' functions required to generate moves. We will do this in 4 demonstration stages.

1. <b>Random moves</b> - random moves made from a list of legal moves

2. **Best move available** - the move that maximises the board evaluation, with little foresight as to what will happen next.

3. <b>Minimax algorithm</b> - algorithm minimises or maximises the moves at each layer of the potential 'move tree' up until a specified depth. For this we will need a function to evaluate the board and the minmax algorithm (move search) algorithm.

4. <b>Minimax algorithm with alpha-beta pruning</b> - similar to above however the algorithm 'prunes' the sections of the move tree which will never be taken as it is already worse than another section of the tree. This cuts down on computing and thus allows for deeper search for the same computing time.

Let us now create some functions to display the board and input our moves.

In [13]:
def human_input(board):
    """ Allows human input by typing in the desired move in UCI protocol."""
    input_move = input("State your move: ")
    
    promotion_input_move = input_move + "q"
    
    first_choice = chess.Move.from_uci(promotion_input_move)
    
    second_choice = chess.Move.from_uci(input_move)
    
    move_list = [] 
    for move in board.legal_moves:
        move_list.append(move)
    
    if first_choice in move_list:
        board.push(first_choice)
    
    elif second_choice in move_list:
        board.push(second_choice)
    
    else:
        print("Invalid move")
        human_input(board)
    
def game_over(board):
    """Returns True if board is in a finished state."""
    return chess.Board.is_game_over(board)

print("Cell has been run.")

Cell has been run.


Random move generation is fairly simple:

In [23]:
def make_random_move(board):
    """Makes a random move on the board that is input. Returns nothing."""
    move_list = []
    for move in board.legal_moves:
        move_list.append(move)
    move = random.choice(move_list)
    board.push(move)

def play_random_comp_v_comp(board):
    """Pits computer vs computer in a randomly chosen move game.
    Recursive function to allow only one move at a time so that the game can be followed easily."""
    if game_over(board) == True:
        print("Game is over.")
        print(chess.Board.result(board))
        return
    make_random_move(board)
    display(board)
    enter = input("Press enter for the next move...")
    play_random_comp_v_comp(board)

def play_random_human_v_comp(board):
    """Pits human vs computer in a randomly chosen move game.
    Recursive function to allow only one move at a time so that the game can be followed easily."""
    if game_over(board) == True:
        print("Game is over.")
        print(chess.Board.result(board))
        return
    human_input(board)
    display(board)
    make_random_move(board)
    display(board)
    play_random_human_v_comp(board)

print("Cell has been run.")

Cell has been run.


# <b>Demonstration.</b>  
Computer vs Computer. To rerun the cell, stop it first then run again.

In [ ]:
board = chess.Board()
display(board)

play_random_comp_v_comp(board)

# <b>Demonstration.</b> 
Human vs Computer. When asked for human input, simply write the square you want to move from followed by the square you want to move to. E.g. c2c4 will move the c2 pawn to c4 etc

In [ ]:
board = chess.Board()
display(board)

play_random_human_v_comp(board)

It is quite awful at chess. We will now integrate some chess strategy into the choice of move through a simple evaluation function. This just runs through the board and adds, if it is the players piece, or subtracts, if the opposition's, the piece value to a total. We will make the computer pick the move the maximises its board evaluation.

In [22]:
def piece_value(piece = int):
    """Assigns a value to each type of piece. 1 - Pawn, 2 - Bishop, 3 - Knight, 4 - Rook, 5 - Queen, 6 - King.
    These representations are from the PyChess documentation."""
    if piece == 1:
        return 100
    elif piece == 2:
        return 350
    elif piece == 3:
        return 350
    elif piece == 4:
        return 525
    elif piece == 5:
        return 1000
    elif piece == 6:
        return 10000
    else:
        return 0        #When there is no piece at that square
    
def evaluate_board(board,player_colour = bool):
    """Evaluates the board by running over every square and adding the piece value if it is the player_colour,
    and subtracting if not. Returns the aggregate value."""
    
    value = 0
    
    for square_num in range(0,64):
        
        piece_type = chess.Board.piece_type_at(board,square_num)
        
        piece_colour = chess.Board.color_at(board,square_num)
        
        if player_colour == piece_colour:
            value += piece_value(piece_type)
        else:
            value -= piece_value(piece_type)
            
    return value

def calculate_best_move(board,player_colour= bool):
    """Runs through a list of possible moves and RETURNS the move that increases the board evaluation the most.
    If two or more moves give the same evaluation, a random move is chosen."""
    
    possible_move_list = []
    for move in board.legal_moves:
        possible_move_list.append(move)
    
    random.shuffle(possible_move_list)  #randomises move choice so the same situation will give rise to different consecutive moves if there is an equal evaluation
        
    best_move = None    
    
    current_best_evaluation = float("-inf")
    
    for move in possible_move_list:
        
        board.push(move)
                
        new_evaluation = evaluate_board(board,player_colour)
        
        if new_evaluation >= current_best_evaluation:
            
            best_move = move
            
            current_best_evaluation = new_evaluation
            
            board.pop()
        
        else:
            
            board.pop()
            
    return best_move

def play_computer_v_computer_oneeval(board):
    """Pits computer against computer in this max evaluation game. Also allows for one move at a time."""
    if game_over(board) == True:
        print("Game is over.")
        print(chess.Board.result(board))
        return
    board.push(calculate_best_move(board,True))
    display(board)
    enter = input("Press enter for the next move...")
    play_computer_v_computer_oneeval(board) 
    
print("Cell has been run.")

Cell has been run.


# **Demonstration.**

In [ ]:
board = chess.Board()
display(board)

play_computer_v_computer_oneeval(board)

Now to improve the 'intelligence' of the computer we will employ a minimax algorithm.

In [25]:
def minimax(board, player_maximising, is_max_turn, depth=2):
    """Searches for the best possible move for a given search depth. It returns the board evaluation 
    and move to be pushed to the board."""
    
    #Once we've hit 0, we can stop
    
    if depth == 0:
        
        return evaluate_board(board,player_maximising), None
    
    best_value = float("-inf") if is_max_turn else float("inf")
    
    best_move = None
    
    possible_move_list = []
    for move in board.legal_moves:
        possible_move_list.append(move)
    random.shuffle(possible_move_list)
    
    for move in possible_move_list:
        
        board.push(move)
        
        current_eval, current_move = minimax(board, depth-1, player_maximising, not is_max_turn)
        
        if is_max_turn and best_value < current_eval:  #if it is the max players turn, maximise the value
            
            best_value = current_eval
            
            best_move = move
        
        elif is_max_turn == False and best_value > current_eval:                #If min players turn, minimise the value
            
            best_value = current_eval
            
            best_move = move
            
        board.pop()
    
    return best_value, best_move

def play_comp_v_comp_minimax(board,depth):
    """Pits computer v computer using the minimax algorithm."""
        if game_over(board) == True:
            print("Game is over.")
            print(chess.Board.result(board))
            return
        value, move = minimax(board,True,True,depth=depth)    #Player colour is a bool, True = White
        board.push(move)
        display(board)
        
        enter = input("Press enter for the next move...")
        
        if game_over(board) == True:
            print("Game is over.")
            print(chess.Board.result(board))
            return
        
        value, move = minimax(board,False,True,depth=depth)
        board.push(move)
        display(board)
        
        enter = input("Press enter for the next move...")
        
        play_comp_v_comp_minimax(board,depth)

print("Cell has been run.")

Cell has been run.


# **Demonstration.** 
The depth can be changed, the deeper it searches the longer the computing time but the better the chess intelligence.

In [ ]:
board = chess.Board()
display(board)

depth = 3

play_comp_v_comp_minimax(board,depth)

Now with alpha-beta pruning we will be able to get deeper for the same computational cost.

In [29]:
def minimax_alphabeta(board, depth = int, player_maximising = bool, alpha = float("-inf"), beta = float("inf"), is_max_turn = True):
    """Uses minimax search algorithm with alpha beta pruning to cut sections of the move tree that will 
    never be chosen as a better move has already been found. RETURNS the best move value and best move."""
    #Once we've hit 0, we can stop
    
    if depth == 0:
        
        return evaluate_board(board,player_maximising), None
    
    best_value = float("-inf") if is_max_turn else float("inf")
    
    best_move = None
    
    possible_move_list = []
    for move in board.legal_moves:
        possible_move_list.append(move)
    random.shuffle(possible_move_list)
    
    for move in possible_move_list:
        
        board.push(move)
        
        current_eval, current_move = minimax_alphabeta(board, depth-1, player_maximising, alpha, beta, not is_max_turn)
        
        if is_max_turn and best_value < current_eval:  #if it is the max players turn, maximise the value
            
            best_value = current_eval
            
            best_move = move
            
            alpha = max(alpha, best_value)
            
            if beta <= alpha:
                
                board.pop()
                
                break
        
        elif is_max_turn == False and best_value > current_eval:                #If min players turn, minimise the value
            
            best_value = current_eval
            
            best_move = move
            
            beta = min(beta,best_value)
            
            if beta <= alpha:
                
                board.pop()
                
                break
            
        board.pop()
    
    return best_value, best_move

def play_comp_v_comp_minimax_alphabeta(board,depth=4):
        """Pits computer v computer using the minimax algorithm with alpha beta pruning"""
        if game_over(board) == True:
            print("Game is over.")
            print(chess.Board.result(board))
            return
        value,move = minimax_alphabeta(board,depth,True)
        board.push(move)
        display(board)
        
        enter = input("Press enter for the next move...")
        
        if game_over(board) == True:
            print("Game is over.")
            print(chess.Board.result(board))
            return
        
        value, move = minimax_alphabeta(board,depth,False)
        board.push(move)
        display(board)
        
        enter = input("Press enter for the next move...")
        
        play_comp_v_comp_minimax_alphabeta(board,depth)
        
print("Cell has been run.")

Cell has been run.


# Demonstration.

Finally has some chess intelligence and plays a very equal game against itself. It stills struggles with end game move progressions which generally require more thought than 4 moves ahead. For example, in a rook ending the rooks move round them board to different positions with little idea what to do. Having said that, many humans are the same!

In [ ]:
board = chess.Board()
display(board)

depth = 4

play_comp_v_comp_minimax_alphabeta(board,depth)

# Possible future improvements

1. **GUI**

Interfacing with the current set up wouldn't enjoyable if you actually wanted to play a game. We would need a way to use the mouse to drag pieces. One way of doing this is to integrate a well known chess gui such as XBoard or UCI using the protocols that PyChess employs. I could also create the data structures for the chess pieces and rules myself which would make it easier to set up an interface with pygame. This would be much more work, however, to obtain the same functionality as PyChess.

2. **Search algorithm**

To improve the eficiency of the pruning we could order the moves such that the more likely to be 'better' moves are evaluated first, such as pawn take moves. Particularly ordering the moves near to first move as it is relatively cheaper to sort these than the exponentially increasing number of moves deeper into the move tree.

3. **Evaluation**

We could also improve the evaluation algorithm. Firstly, by adding in extra value for pieces in certain positions. For isntance, having pieces in the middle of the board is beneficial. Particularly knights and pawns. We could also add in different evaluations for different parts of the game. A bishop in the end game is far more valuable than beginning/middle game.

4. **Move analysis**

Could also incorporate move analysis from an exiting chess engine such as stockfish. Could do a game analysis at the end of a game. The core of the intelligence though will remain as I made it, it could just be a way of quantifying how good the intelligence is compared to professional engines.